In [1]:
!pip install pdf2image pillow paddlepaddle paddleocr opencv-python numpy
!pip install --upgrade tensorboard==2.18.0
!pip install --upgrade tensorflow-tpu
!apt-get install -y poppler-utils
!pip install pdfplumber
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install pytesseract
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 16.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=d8fcfc2d91d9d5ec20e5bc2deafbc787eb0cf88a3eb4297d0939813c3a8a808d
  Stored in

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

In [3]:
### direct ocr on that page, has image based table

from pdf2image import convert_from_path
import os
import pdfplumber
import pytesseract
from PIL import Image

# Function to check if a page contains a table (using pdfplumber for digital PDFs)
def contains_table_with_pdfplumber(pdf_path, page_number):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]
        # Extract tables on the page
        tables = page.extract_tables()
        return len(tables) > 0

# Function to check if a page contains a table (using OCR for image-based PDFs)
def contains_table_with_ocr(pdf_path, page_number):
    # Convert the specific page to an image
    pages = convert_from_path(pdf_path, 300, first_page=page_number+1, last_page=page_number+1)
    page_image = pages[0]

    # Use OCR to detect text and check for potential table-like patterns
    text = pytesseract.image_to_string(page_image)

    # Check if the text contains common table delimiters (like "|", or patterns like "Row", "Column", etc.)
    if "|" in text or "Row" in text or "Column" in text:
        return True
    return False

# Function to convert PDF pages with tables (either digital or image-based) to images
def pdf_to_images_with_tables(pdf_path, output_folder):
    # Open the PDF using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all pages and check for tables
    for i in range(total_pages):
        # First check if page contains a table using pdfplumber (for digital text PDFs)
        if contains_table_with_pdfplumber(pdf_path, i):
            pages = convert_from_path(pdf_path, 300, first_page=i+1, last_page=i+1)
            for page in pages:
                page.save(f'{output_folder}/page_{i+1}.png', 'PNG')
            print(f"Page {i+1} contains a table (digital), converted to image and saved.")

        # If no table found, check using OCR for image-based tables
        elif contains_table_with_ocr(pdf_path, i):
            pages = convert_from_path(pdf_path, 300, first_page=i+1, last_page=i+1)
            for page in pages:
                page.save(f'{output_folder}/page_{i+1}.png', 'PNG')
            print(f"Page {i+1} contains a table (image-based), converted to image and saved.")

        else:
            print(f"Page {i+1} does not contain a table, skipped.")

# Example usage
pdf_path = '/content/Reliance.pdf'  # Path to your PDF
output_folder = '/content/output_images'  # Folder to save images
pdf_to_images_with_tables(pdf_path, output_folder)


Page 1 does not contain a table, skipped.
Page 2 does not contain a table, skipped.
Page 3 does not contain a table, skipped.
Page 4 does not contain a table, skipped.
Page 5 does not contain a table, skipped.
Page 6 does not contain a table, skipped.
Page 7 does not contain a table, skipped.
Page 8 does not contain a table, skipped.
Page 9 contains a table (image-based), converted to image and saved.
Page 10 contains a table (digital), converted to image and saved.
Page 11 contains a table (digital), converted to image and saved.
Page 12 does not contain a table, skipped.
Page 13 contains a table (digital), converted to image and saved.
Page 14 does not contain a table, skipped.
Page 15 contains a table (digital), converted to image and saved.
Page 16 contains a table (digital), converted to image and saved.
Page 17 does not contain a table, skipped.
Page 18 does not contain a table, skipped.
Page 19 does not contain a table, skipped.
Page 20 contains a table (digital), converted to i

In [4]:
#### preprocess_images.py (for not sequentioal images name like page 4 then page 9 so on)

import os
import cv2
import numpy as np

# Function to preprocess images
def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Deskew the image (if skewed)
    gray = deskew_image(gray)

    # Noise removal with Gaussian Blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Binarization (Thresholding)
    _, binarized = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Dilation (to connect broken text)
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(binarized, kernel, iterations=1)

    return dilated

# Function to deskew the image
def deskew_image(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = 90 - angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Example usage
image_folder = '/content/output_images'
preprocessed_folder = '/content/preprocessed_images'

# Create the preprocessed folder if it doesn't exist
if not os.path.exists(preprocessed_folder):
    os.makedirs(preprocessed_folder)

# List all files in the image folder
files = os.listdir(image_folder)

# Check if there are any files in the folder
if not files:
    print(f"No files found in {image_folder}. Please check the folder.")
else:
    # Process each image file found in the folder
    for file in files:
        image_path = os.path.join(image_folder, file)

        if image_path.endswith(".png"):  # Process only PNG files
            print(f"Processing {file}...")
            processed_image = preprocess_image(image_path)
            cv2.imwrite(os.path.join(preprocessed_folder, f"preprocessed_{file}"), processed_image)
            print(f'Preprocessed {file} saved.')

        else:
            print(f"Skipping {file}, not a PNG image.")


Processing page_25.png...
Preprocessed page_25.png saved.
Processing page_11.png...
Preprocessed page_11.png saved.
Processing page_23.png...
Preprocessed page_23.png saved.
Processing page_21.png...
Preprocessed page_21.png saved.
Processing page_20.png...
Preprocessed page_20.png saved.
Processing page_26.png...
Preprocessed page_26.png saved.
Processing page_15.png...
Preprocessed page_15.png saved.
Processing page_9.png...
Preprocessed page_9.png saved.
Processing page_16.png...
Preprocessed page_16.png saved.
Processing page_13.png...
Preprocessed page_13.png saved.
Processing page_10.png...
Preprocessed page_10.png saved.


In [ ]:
#### data conversion to json and csv format

import cv2
import numpy as np
import easyocr
import pandas as pd
import os
import json

# Directory path containing images {original}
image_dir = '/content/preprocessed_images/'

# Initialize EasyOCR
reader = easyocr.Reader(['en'])

# Loop through all files in the directory
for image_filename in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_filename)

    # Check if it's an image file
    if image_filename.lower().endswith(('png', 'jpg', 'jpeg')):
        # Load the image
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error: Unable to read image {image_filename}")
            continue

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)

        # Detect horizontal and vertical lines using Hough Line Transform
        # Horizontal lines detection
        horizontal_lines = cv2.HoughLinesP(thresh, 1, np.pi / 180, threshold=150, minLineLength=100, maxLineGap=10)

        # Vertical lines detection
        vertical_lines = cv2.HoughLinesP(thresh, 1, np.pi / 180, threshold=150, minLineLength=100, maxLineGap=10)

        # Create an empty mask to draw the lines on the image
        line_image = np.zeros_like(image)

        if horizontal_lines is not None:
            for line in horizontal_lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 2)

        if vertical_lines is not None:
            for line in vertical_lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Combine the original image with the lines drawn
        combined_image = cv2.addWeighted(image, 0.8, line_image, 1, 0)

        # Use EasyOCR to extract text from the table cells
        text = reader.readtext(combined_image, detail=0)

        # Process extracted text
        table_data = []
        for row in text:
            cleaned_row = row.replace(',', '').strip()  # Remove commas and strip spaces
            if cleaned_row:
                table_data.append(cleaned_row.split())

        # Ensure uniform column count
        max_columns = max(len(row) for row in table_data) if table_data else 0
        structured_table = [row + [''] * (max_columns - len(row)) for row in table_data]

        # Convert to Pandas DataFrame
        df = pd.DataFrame(structured_table)

        # Save as CSV (table format)
        output_csv = f"extracted_table_data_{image_filename}_.csv"
        df.to_csv(output_csv, index=False, header=False)
        print(f"Table extracted successfully from {image_filename} and saved as {output_csv}")

        # Convert to JSON format with structured columns
        json_output = {}
        if structured_table:
            headers = structured_table[0]  # Assume first row as headers
            column_data = {header: [] for header in headers}

            for row in structured_table[1:]:  # Skip the header row
                for i, value in enumerate(row):
                    column_data[headers[i]].append(value)

            json_output = column_data  # Assign column-wise data

        # Save as JSON
        output_json = f"extracted_table_data_{image_filename}.json"
        with open(output_json, "w", encoding="utf-8") as json_file:
            json.dump(json_output, json_file, indent=4)

        print(f"JSON extracted successfully from {image_filename} and saved as {output_json}")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteTable extracted successfully from preprocessed_page_23.png and saved as extracted_table_data_preprocessed_page_23.png_.csv
JSON extracted successfully from preprocessed_page_23.png and saved as extracted_table_data_preprocessed_page_23.png.json
Table extracted successfully from preprocessed_page_10.png and saved as extracted_table_data_preprocessed_page_10.png_.csv
JSON extracted successfully from preprocessed_page_10.png and saved as extracted_table_data_preprocessed_page_10.png.json
Table extracted successfully from preprocessed_page_13.png and saved as extracted_table_data_preprocessed_page_13.png_.csv
JSON extracted successfully from preprocessed_page_13.png and saved as extracted_table_data_preprocessed_page_13.png.json


In [ ]:
import json
from tabulate import tabulate

# Load JSON data from file
file_path = "extracted_table_data_preprocessed_page_10.png.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Define keys to extract
keys_to_extract = ["Revenue", "Profit", "Cost", "Purchases"]

# Extract values for the required keys
extracted_data = {key: [] for key in keys_to_extract}
for key in keys_to_extract:
    if key in data["Name"]:
        index = data["Name"].index(key)
        extracted_data[key] = data["Name"][index + 1 : index + 6]  # Assuming five values per key

# Convert extracted values to integers (if possible)
for key, values in extracted_data.items():
    extracted_data[key] = [int(value) for value in values if value.isdigit()]

# Define the custom period labels
period_labels = [
    "Quarter ended (31 Dec'24)",
    "Quarter ended (30 Sep'24)",
    "Quarter ended (31 Dec'23)",
    "Nine month ended (31 Dec'24)",
    "Nine month ended (31 Dec'23)",
    "Year ended (31 Mar'24)"
]

# Add row labels based on periods
table_data = [[period_labels[i]] + list(row) for i, row in enumerate(zip(*[extracted_data[key] for key in keys_to_extract]))]
headers = [""] + keys_to_extract

# Print formatted table
print(tabulate(table_data, headers=headers, tablefmt="grid"))
